In [1]:
import os
import time, datetime
import config
import connection_tools
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import json

In [2]:
connection = connection_tools.connect_db()
cur = connection.cursor()

In [3]:
lol_watcher = LolWatcher(os.getenv('RIOT_TOKEN'))
region = "jp1"
large_region = "ASIA"
season_start = 1637625600
season_end = ""
queue = "RANKED_SOLO_5x5"
queue_id = 420

In [4]:
match_list = []

#Challenger
summoners = lol_watcher.league.challenger_by_queue(region, queue)["entries"]
for summoner in summoners:
    summonerName = summoner["summonerName"]
    try:
        puuid = lol_watcher.summoner.by_name(region, summonerName)["puuid"]
        match_list+=lol_watcher.match.matchlist_by_puuid(region=large_region, puuid=puuid, queue=queue_id, start_time=season_start)
    except:
        continue
    time.sleep(2)

#Grandmaster
summoners = lol_watcher.league.grandmaster_by_queue(region, queue)["entries"]
for summoner in summoners:
    summonerName = summoner["summonerName"]
    try:
        puuid = lol_watcher.summoner.by_name(region, summonerName)["puuid"]
        match_list+=lol_watcher.match.matchlist_by_puuid(region=large_region, puuid=puuid, queue=queue_id, start_time=season_start)
    except:
        continue
    time.sleep(2)

#Master
summoners = lol_watcher.league.masters_by_queue(region, queue)["entries"]
for summoner in summoners:
    summonerName = summoner["summonerName"]
    try:
        puuid = lol_watcher.summoner.by_name(region, summonerName)["puuid"]
        match_list+=lol_watcher.match.matchlist_by_puuid(region=large_region, puuid=puuid, queue=queue_id, start_time=season_start)
    except:
        continue
    time.sleep(2)

match_list = set(match_list)

In [5]:
match_list_old = []
cur.execute("select match_id from match_data")
for c in cur:
    match_list_old.append(c[0])

for match_id in match_list:
    if match_id not in match_list_old:
        match_info = lol_watcher.match.by_id(region=large_region, match_id=match_id)
        time.sleep(2)
        match_ready = match_info["info"]["teams"]
        match_result = pd.DataFrame(match_info["info"]["participants"])
        match_timeline = lol_watcher.match.timeline_by_match(region=large_region, match_id=match_id)
        time.sleep(2)
        match_date = match_info["info"]["gameCreation"]
        match_champions = ""
        for champion in match_result["championName"]:
            match_champions += "'" + champion + "', "
        match_champions = match_champions[:-2]
        match_version = match_info["info"]["gameVersion"]
        cur.execute("insert into match_data values ('%s', %s, '%s', '%s', '%s', '%s', '%s', %s);" % (match_id, match_date, match_version, large_region, region, json.dumps(match_info), json.dumps(match_timeline), match_champions))
    connection.commit()

In [ ]:
match_result = pd.DataFrame(match_info["info"]["participants"])
match_players = pd.merge(pd.DataFrame(match_timeline["info"]["participants"]), match_result[["participantId", "puuid"]], suffixes=["_timeline", "_result"], on="puuid")
match_info2 = match_info["info"].copy()
for c in ["participants", "teams"]:
    match_info2.pop(c)

NameError: name 'match_timeline' is not defined

In [ ]:
game_events = [[] for x in range(21)]
frames = match_timeline["info"]["frames"]
for frame in frames[1:-1]:
    events = frame["events"]
    for event in events:
        if "participantId" in list(event.keys()):
            playerId = event["participantId"]
            game_events[playerId].append(event)
        
        elif "creatorId" in list(event.keys()):
            playerId = event["creatorId"]
            game_events[playerId].append(event)
        
        else:
            game_events[0].append(event)

In [ ]:
types = []
for frame in frames[1:-1]:
    events = frame["events"]
    for event in events:
        types.append(event["type"])